In [1]:
import numpy as np
import os
import fitsio as fio
import healpy as hp
import src.catalog as catalog
import src.config as config
import src.fig as fig
import src.lin as lin
import src.sys_split as sys_split
import src.corr as corr
import src.field as field
import src.pz as pz
import src.cosmo as cosmo
import src.y1shearcat as y1
import src.txt as txt
import cPickle
import scipy.stats

No mpi4py


In [2]:
#This lets you view PDF images in the browser
class PDF(object):
  def __init__(self, pdf, size=(800,800)):
    self.pdf = pdf
    self.size = size

  def _repr_html_(self):
    return '<iframe src={0} width={1[0]} height={1[1]}></iframe>'.format(self.pdf, self.size)

  def _repr_latex_(self):
    return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.pdf)

In [3]:
goldfile  = '/global/cscratch1/sd/troxel/finaly1cats/y1a1-gold-mof-badregion.fits'
i3file    = '/global/cscratch1/sd/troxel/finaly1cats/y1a1-im3shape_v5_matched_v6.fits'
mcalfile  = '/global/cscratch1/sd/troxel/finaly1cats/mcal-y1a1-combined-riz-blind-v4-matched.fits'
i3pickle  = '/global/cscratch1/sd/troxel/finaly1cats/i3.cpickle'
mcalpickle= '/global/cscratch1/sd/troxel/finaly1cats/mcal.cpickle'
bpzfile   = '/global/cscratch1/sd/troxel/finaly1cats/BPZ_ngmix_mof_slr_HiZ_combined_matched.fits'
bpz0file  = '/global/cscratch1/sd/troxel/finaly1cats/BPZ_ngmix_sheared_matched.fits'

i3epochdir= '/project/projectdirs/des/wl/desdata/wlpipe/im3shape_y1a1_v5/bord/epoch/'
mcalepochdir = '/global/cscratch1/sd/troxel/finaly1cats/'
i3epochpickle  = '/global/cscratch1/sd/troxel/finaly1cats/i3epoch.cpickle'
mcalepochpickle= '/global/cscratch1/sd/troxel/finaly1cats/mcalepoch.cpickle'
imagefile = '/global/cscratch1/sd/troxel/finaly1cats/y1a1_image_id.fits'


In [12]:
mcalepoch,i3epoch = y1.y1.load_epoch_data(i3epochdir,mcalepochdir,imagefile,i3pickle,mcalpickle,i3epochpickle,mcalepochpickle)
mcalepoch.ccd=mcalepoch.ccd.astype(int)-1
i3epoch.ccd=i3epoch.ccd.astype(int)-1

In [18]:
def make_fov_data(i3epoch,mcalepoch):
    import scipy.stats
    mcal_e1=[]
    mcal_e2=[]
    bins=(5,10)
    for ccd in xrange(62):
        w=np.where(mcalepoch.ccd==ccd)
        if len(w[0])==0:
            e1=np.zeros(bins)/0.0
            e2=np.zeros(bins)/0.0
        else:
            row=mcalepoch.row[w]
            col=mcalepoch.col[w]
            e1=mcalepoch.e1[w]
            e2=mcalepoch.e2[w]
            e1, _, _, _ = scipy.stats.binned_statistic_2d(row,col,e1, bins=bins)
            e2, _, _, _ = scipy.stats.binned_statistic_2d(row,col,e2, bins=bins)    
        mcal_e1.append(e1)
        mcal_e2.append(e2)

    i3_e1=[]
    i3_e2=[]
    for ccd in xrange(62):
        w=np.where(i3epoch.ccd==ccd)
        if len(w[0])==0:
            e1=np.zeros(bins)/0.0
            e2=np.zeros(bins)/0.0
        else:
            row=i3epoch.row[w]
            col=i3epoch.col[w]
            e1=i3epoch.e1[w]
            e2=i3epoch.e2[w]
            e1, _, _, _ = scipy.stats.binned_statistic_2d(row,col,e1, bins=bins)
            e2, _, _, _ = scipy.stats.binned_statistic_2d(row,col,e2, bins=bins)    
        i3_e1.append(e1)
        i3_e2.append(e2)
    fov_data = mcal_e1, mcal_e2, i3_e1, i3_e2
    return fov_data

In [19]:
fov_data = make_fov_data(i3epoch,mcalepoch)
cPickle.dump(fov_data, open("text/fov_data_10.pickle", "w"))

In [4]:
fov_data = cPickle.load(open("text/fov_data_10.pickle"))

In [9]:
mcal_e1, mcal_e2, i3_e1, i3_e2 = fov_data


In [20]:
def make_fov_plot(fov_data, cmap, filename):
    from matplotlib import transforms
    rot = transforms.Affine2D().rotate_deg(-90)

    mcal_e1, mcal_e2, i3_e1, i3_e2 = fov_data
    import matplotlib.pyplot as plt
    import matplotlib.gridspec as gridspec
    fig=plt.figure(figsize=(8,8))
    gs = gridspec.GridSpec(8, 9)
    mcal_e1_ax=gs[0:4,0:4]
    mcal_e2_ax=gs[4:8,0:4]
    i3_e1_ax=gs[0:4,4:8]
    i3_e2_ax=gs[4:8,4:8]
    gs2 = gridspec.GridSpec(8, 18)
    cbar_ax=ax=plt.subplot(gs2[2:6,17])

    def do_plane(E,spec):
        ax=plt.subplot(spec)
        ax.set_frame_on(False)
        ax.set_xticks([])
        ax.set_yticks([])

        corners=field.field_methods.ccd_corners()
        for c,e in zip(corners,E):
            left=c[0][1]
            right=c[1][1]
            bottom=c[0][0]
            top=c[2][0]
            extent=((left,right,bottom,top))
            extent=((bottom,top,-right,-left))
            im=ax.imshow(e.T,extent=extent,interpolation='nearest',vmin=-0.004,vmax=0.004, cmap=cmap)
        plt.xlim(-202,202)
        plt.ylim(-220,220)
        return ax,im


    ax,im=do_plane(mcal_e1, mcal_e1_ax)
    ax.set_ylabel("$e_1$", labelpad=25)

    ax,im=do_plane(mcal_e2, mcal_e2_ax)
    ax.set_title(r"{\sc Metacal}", variant='small-caps', family='serif')
    ax.set_ylabel("$e_2$", labelpad=25)

    ax,im=do_plane(i3_e1, i3_e1_ax)

    ax,im=do_plane(i3_e2, i3_e2_ax)
    ax.set_title(r"{\sc Im3shape}", variant='small-caps', family='serif')

    plt.colorbar(im,cax=cbar_ax)
    plt.savefig(filename)
    plt.close()

In [21]:
make_fov_plot(fov_data,'plasma', 'plots/y1/mean_e_fov.pdf')
#Image(filename='tmp.png') 

In [22]:
PDF("plots/y1/mean_e_fov.pdf")